In [1]:
import requests
from bs4 import BeautifulSoup
from astropy.io import fits
import matplotlib.pyplot as plt
from desispec.io.spectra import read_spectra
import numpy as np
from desitarget.targetmask import desi_mask
from astropy.table import Table

In [2]:
r_i = requests.get("https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/") 
soup_i = BeautifulSoup(r_i.content, "html.parser")
all_links_i = soup_i.find_all("a")
linkList_i = [link['href'] for link in all_links_i[1:]] # creates a list of all the first indicies of the coaad files

all_lists = []
for i in range(len(linkList_i)):
    r_j = requests.get("https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/" + str(linkList_i[i]))
    soup_j = BeautifulSoup(r_j.content, "html.parser")
    all_links_j = soup_j.find_all("a")
    linkList_j = [link['href'] for link in all_links_j] 
    all_lists.append(linkList_j[1:]) # creates a list of all the second indicies of the coaad files for each of the first indicies

In [ ]:
hdul = fits.open("qso_cat_dr1_main_dark_healpix_zlya-v0.fits") # lya quasar catalog
data = hdul[1].data
flux_b = []; flux_r = []; flux_z = [] # Initialize lists
wave_b = []; wave_r = []; wave_z = []
ivar_b = []; ivar_r = []; ivar_z = []
zArray = []
tidArray = []
for i in range(1): #range(len(linkList_i))
    for j in range(2): #range(len(all_lists[i]))
        r = requests.get("https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/healpix/main/dark/" +  str(linkList_i[i]) # uses url generated from linkList and all_lists
                     + str(all_lists[i][j]) + "coadd-main-dark-"+  str(all_lists[i][j]).replace("/","") + ".fits")
        print(r.status_code)
        with open("coadd-main-dark-" + str(all_lists[i][j]).replace("/","") + ".fits", "wb") as fp: #dowloads the file with the name generated from all_lists
            fp.write(r.content)
            
        spec = read_spectra("coadd-main-dark-" + str(all_lists[i][j]).replace("/","") + ".fits" ) # gets spectra data from the coaad file
        fibermap = spec.fibermap
        coadd_dict = {tid: ii for ii, tid in enumerate(fibermap["TARGETID"])} # creates a dictionary of indicies with the target id as the key
        validQSOidx = []
        for ii, tid in enumerate(data["TARGETID"]):
            if (data["Z"][ii] > 2) and (data["ZWARN"][ii] == 0) and (data["BI_CIV"][ii] == 0): #checks that Z > Z_min, ZWARN = 0, and BI_CIV > BI_CIV_min
                if tid in coadd_dict: # checks that the target id from the lya QSO catelog is in the coadd file
                    tidArray.append(tid) #generates a list of target id's of valis QSOs
                    zArray.append(data["Z"][ii]) # generates a list of redshift of all the valid QSOs
                    wave_b = spec.wave['b'] # extracts wavelegnth for all three arms
                    wave_r = spec.wave['r']
                    wave_z = spec.wave['z']
                    wave = np.concatenate((wave_b,wave_r,wave_z)) # concatenates the three arms
                    flux_b = spec.flux['b'][coadd_dict[tid]]
                    flux_r = spec.flux['r'][coadd_dict[tid]]
                    flux_z = spec.flux['z'][coadd_dict[tid]]
                    flux = np.concatenate((flux_b,flux_r,flux_z))
                    ivar_b = spec.flux['b'][coadd_dict[tid]]
                    ivar_r = spec.flux['r'][coadd_dict[tid]]
                    ivar_z = spec.flux['z'][coadd_dict[tid]]
                    ivar = np.concatenate((ivar_b,ivar_r,ivar_z))
                    t = Table()
                    t['WAVE'] = wave # formats the datat into a table
                    t['FLUX'] = flux
                    t['IVAR'] = ivar
                    t.write(str(tid) + '.fits') # writes the data to a fits file for analysis

                    
        filename = "coadd-main-dark-" + str(all_lists[i][j]).replace("/", "") + ".fits"
        !rm -rf "{filename}" # removes the downloaded coadd file


200
INFO:spectra.py:451:read_spectra: iotime 6.368 sec to read spectra from:  coadd-main-dark-0.fits at 2025-06-25T11:39:30.055631
200
INFO:spectra.py:451:read_spectra: iotime 6.468 sec to read spectra from:  coadd-main-dark-1.fits at 2025-06-25T11:42:18.518696


In [10]:
print(wave_b)
print(wave_r)
print(np.concatenate((wave_b,wave_r,wave_z)))

[3600.  3600.8 3601.6 ... 5798.4 5799.2 5800. ]
[5760.  5760.8 5761.6 ... 7618.4 7619.2 7620. ]
[3600.  3600.8 3601.6 ... 9822.4 9823.2 9824. ]
